# Lyrics data fetching

### Import relevant packages

In [25]:
import pandas as pd
import lyricsgenius as lg
import re
import json
import time

### Load artist data

In [3]:
ARTIST_PATH = "/Users/pascalessig/CBS - Copenhagen Business School/DSBA Group - General/Data/Top_1000_Artists.csv"

In [4]:
artists = pd.read_csv(ARTIST_PATH, index_col=0)

In [5]:
artists.head()

,Artist Name,Streams,Tracks,1b+,100m+,10m+,1m+,Last Update
1,Drake,"34,422,345,387",223,3,90,208,219,11.06.20
2,Ed Sheeran,"26,548,511,656",204,5,49,140,174,11.06.20
3,Post Malone,"21,650,426,333",64,8,47,57,60,11.06.20
4,Eminem,"20,367,427,868",265,1,46,197,239,11.06.20
5,Ariana Grande,"20,209,530,076",181,2,46,107,157,11.06.20


In [9]:
artists.iloc[:50, 0]

1                 Drake
2            Ed Sheeran
3           Post Malone
4                Eminem
5         Ariana Grande
6         Justin Bieber
7            The Weeknd
8              J Balvin
9             Bad Bunny
10           Kanye West
11               Khalid
12                Ozuna
13             Coldplay
14              Rihanna
15         XXXTENTACION
16      Imagine Dragons
17         Taylor Swift
18         Travis Scott
19             Maroon 5
20        Billie Eilish
21        Calvin Harris
22         Shawn Mendes
23         David Guetta
24                  BTS
25         Daddy Yankee
26     The Chainsmokers
27            Sam Smith
28       Kendrick Lamar
29           Juice WRLD
30             Dua Lipa
31                Queen
32           Bruno Mars
33              Beyoncé
34               Future
35          Chris Brown
36          Nicki Minaj
37               Halsey
38                  Sia
39        One Direction
40         Selena Gomez
41            Lady Gaga
42    Twenty One

### Setting up API

In [6]:
# Note: CLIENT_ID, CLIENT_SECRET and CLIENT_ACCESS_TOKEN were removed to keep them confidential
CLIENT_ID = # Insert CLIENT_ID
CLIENT_SECRET = # Insert CLIENT_SECRET
CLIENT_ACCESS_TOKEN = # Insert CLIENT_ACCESS_TOKEN

In [31]:
def lyrics_fetching(artist_list, access_token, filename, max_songs = 25, sort = "popularity"):
    """
    returns a dataframe with lyrics from artists and with the title accordingly.
    
    Input:
    - artist_list (list): contains the names of the artists
    - access_token (str): token generated based on the genius account
    - filename (str): name for the csv file generated as output
    - max_songs (int): number of songs given out per artist, default setting at 25
    - sort (str): defined by which criteria the songs given out are selected, default set is popularity
    
    Output:
    a dataframe containing the name of the artist, the title of the song, the title with the artist featured, 
    if this does not apply the title from the previous column and lastly the lyrics of the song
    """
    # creating genius object
    genius = lg.Genius(access_token)
    # create df where data is appended into
    df = pd.DataFrame(columns = ["Artist", "Title", "Title_with_featured", "Lyrics"])
    err_ctr = 0
    remaining_artists = len(artist_list)
    # loop through the list of artists fed into the function
    for artist in artist_list:
        while True:
            try:
                # create and object that holds the songs for an artist
                song_list = genius.search_artist(artist, max_songs=max_songs, sort=sort)
             
                # loop through the songs stored in the artist object
                for song in song_list.songs: 
                    # the titles must be accessed through a json format since other format does not allow action
                    df = df.append(pd.DataFrame([[artist, 
                                                  json.loads(song.to_json())["title"], 
                                                  json.loads(song.to_json())["title_with_featured"],
                                                  song.lyrics]],
                                               columns = ["Artist", "Title", "Title_with_featured", "Lyrics"]))
                # when one artist is fully loaded the results overwrite the previous status of the file
                # done to secure progress, even if the code breaks at one point along the way
                df.to_csv(filename)
                # when artist is handled it is deducted and the information printed out
                remaining_artists -= 1
                print(f"\n\n{remaining_artists} artists remaining.\n\n")
                break
                
            except TypeError:
                # if the lyrics could not be loaded in a given time which is defined in the package, error occurs,
                # therefore a exception is implemented to let the code rest for one minute and retry
                sleep = 60
                print(f"\n Exception was thrown for TypeError. Sleeping for {sleep} seconds.\n")
                err_ctr += 1
                time.sleep(sleep)
                pass
            
            except AttributeError:
                print(f"\nException was thrown for AttributeError, resuming with the next artist.\n")
                break
    # reset the index for the final output
    df.reset_index(inplace=True, drop=True)
    print(f"Function has been executed successfully. {err_ctr} errors occurred.")
    return df

In [30]:
# This line illustrates fetching the lyrics of artists with IDs from 150 to 250. In a similar manner,
# all other IDs between 0 and 1,000 were retreived. After retrieving all batches, all files were consolidated
# into one lyrics dataframe
df = lyrics_fetching(artists.iloc[150:250, 0], CLIENT_ACCESS_TOKEN, "150_250.csv")

Searching for songs by Gucci Mane...

Song 1: "I Get the Bag"
Song 2: "Both"
Song 3: "Curve"
Song 4: "Lemonade"
Song 5: "Met Gala"
Song 6: "Nothin On Ya"
Song 7: "Truth (Not a Jeezy Diss)"
Song 8: "Traphouse 3"
Song 9: "St. Brick Intro"
Song 10: "Pussy Print"
Song 11: "Last Time"
Song 12: "I Heard"
Song 13: "1st Day Out tha Feds"
Song 14: "I Think I Love Her"
Song 15: "Back on Road"
Song 16: "Drove U Crazy"
Song 17: "Off the Leash"
Song 18: "Both Eyes Closed"
Song 19: "Both (Remix)"
Song 20: "Guwop Home"
Song 21: "Solitaire"
Song 22: "Dance with the Devil"
Song 23: "Wasted"
Song 24: "Darker"
Song 25: "Hell Yes"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by 6ix9ine...

Song 1: "FEFE"
Song 2: "GOOBA"
Song 3: "GUMMO"
Song 4: "BILLY"
Song 5: "KOODA"
Song 6: "TATI"
Song 7: "KEKE"
Song 8: "STOOPID"
Song 9: "GOTTI"
Song 10: "MAMA"
Song 11: "BEBE"
Song 12: "KIKA"
Song 13: "WAKA"
Song 14: "TIC TOC"
Song 15: "KANGA"
Song 16: "RONDO"
Song 17: "MALA"
Song 18

Song 9: "My Love (Acoustic)"
Song 10: "Saddest Vanilla"
Song 11: "Why Me"
Song 12: "123"
Song 13: "No One"
Song 14: "Broken"
Song 15: "Insecurities"
Song 16: "You Can Find Me"
Song 17: "Right Here"
Song 18: "Thursday (Remix)"
Song 19: "No Rights, No Wrongs"
Song 20: "Rollin"
Song 21: "Hate / Love"
Song 22: "Million Reasons"
Song 23: "Strawberry Fields"
Song 24: "Never Let Me Go"
Song 25: "Love Me"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by Lorde...

Song 1: "Royals"
Song 2: "Green Light"
Song 3: "Liability"
Song 4: "Team"
Song 5: "Ribs"
Song 6: "Tennis Court"
Song 7: "The Louvre"
Song 8: "Hard Feelings/Loveless"
Song 9: "Sober"
Song 10: "Perfect Places"
Song 11: "Homemade Dynamite"
Song 12: "Buzzcut Season"
Song 13: "Writer In the Dark"
Song 14: "Sober II (Melodrama)"
Song 15: "Homemade Dynamite (Remix)"
Song 16: "Supercut"
Song 17: "400 Lux"
Song 18: "Glory and Gore"
Song 19: "White Teeth Teens"
Song 20: "A World Alone"
Song 21: "Yellow Flick

Song 13: "Between Our Hearts"
Song 14: "Boing Boing"
Song 15: "Broccoli"
Song 16: "Don’t Say No"
Song 17: "Ferrari"
Song 18: "Heaven"
Song 19: "Activate"
Song 20: "Washed Up"
Song 21: "The Concept"
Song 22: "Big Butts"
Song 23: "Ahh Yeah"
Song 24: "Home - Tritonal Remix"
Song 25: "Perfect to Be All ’Round Again"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by Mariah Carey...

Song 1: "All I Want for Christmas Is You"
Song 2: "We Belong Together"
Song 3: "#Beautiful"
Song 4: "Obsessed"
Song 5: "Fantasy"
Song 6: "Always Be My Baby"
Song 7: "Hero"
Song 8: "GTFO"
Song 9: "Fantasy (Bad Boy Remix)"
Song 10: "With You"
Song 11: "One Sweet Day"
Song 12: "Touch My Body"
Song 13: "Heartbreaker"
Song 14: "Loverboy"
Song 15: "Shake It Off"
Song 16: "Honey"
Song 17: "Emotions"
Song 18: "Breakdown"
Song 19: "My All"
Song 20: "Without You"
Song 21: "Triumphant (Get ’Em)"
Song 22: "A No No"
Song 23: "Infinity"
Song 24: "Don’t Forget About Us"
Song 25: "Butterfly"


Song 6: "My Mind Is For Sale"
Song 7: "Do You Remember"
Song 8: "Sunsets For Somebody Else"
Song 9: "Home"
Song 10: "Angel"
Song 11: "Breakdown"
Song 12: "Bubble Toes"
Song 13: "Good People"
Song 14: "Never Know"
Song 15: "Taylor"
Song 16: "Belle"
Song 17: "Love Song #16"
Song 18: "Sleep Through The Static"
Song 19: "No Other Way"
Song 20: "I Got You"
Song 21: "Staple It Together"
Song 22: "Only the Ocean"
Song 23: "Gone"
Song 24: "If I Could"
Song 25: "We’re Going To Be Friends"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by Sean Paul...

Song 1: "No Lie"
Song 2: "Temperature"
Song 3: "Get Busy"
Song 4: "Body (Migos Version)"
Song 5: "Like Glue"
Song 6: "I’m Still in Love with You"
Song 7: "Gimme the Light"
Song 8: "Tek Weh Yuh Heart"
Song 9: "We Be Burnin’ (Recognize It)"
Song 10: "She Doesn’t Mind"
Song 11: "(When You Gonna) Give It Up To Me"
Song 12: "Punkie"
Song 13: "Got 2 Luv U"
Song 14: "Entertainment 2.0"
Song 15: "Body"
Song 16: "We Be B

Song 2: "My Way"
Song 3: "That’s Life"
Song 4: "New York, New York"
Song 5: "Somethin’ Stupid"
Song 6: "The Girl From Ipanema (Garota De Ipanema)"
Song 7: "The Way You Look Tonight"
Song 8: "Come Fly with Me"
Song 9: "I’ve Got You Under My Skin"
Song 10: "Have Yourself a Merry Little Christmas"
Song 11: "My Funny Valentine"
Song 12: "It Was a Very Good Year"
Song 13: "Strangers in the Night"
Song 14: "Blue Moon"
Song 15: "Luck Be a Lady"
Song 16: "You Make Me Feel So Young"
Song 17: "Let It Snow! Let It Snow! Let It Snow!"
Song 18: "Autumn Leaves"
Song 19: "In the Wee Small Hours of the Morning"
Song 20: "It Had To Be You"
Song 21: "High Hopes"
Song 22: "The Best Is Yet to Come"
Song 23: "Quiet Nights of Quiet Stars (Corcovado)"
Song 24: "The Lady Is a Tramp"
Song 25: "One for My Baby (And One More for the Road)"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by The Notorious B.I.G....

Timeout raised and caught:
HTTPSConnectionPool(host='api.genius.

Song 18: "Sunrise, Sunburn, Sunset"
Song 19: "Crash My Party"
Song 20: "One Margarita"
Song 21: "I Don’t Want This Night to End"
Song 22: "Do I"
Song 23: "Good Directions"
Song 24: "Rain is a Good Thing"
Song 25: "Roller Coaster"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by Rauw Alejandro...

Song 1: "T.T.I."
Song 2: "Tattoo"
Song 3: "Enchule"
Song 4: "La Oportunidad (Remix)"
Song 5: "Al Mismo Tiempo"
Song 6: "¿Qué Somos?"
Song 7: "Detective"
Song 8: "Yo Sabía"
Song 9: "Algo Mágico"
Song 10: "Borrarte"
Song 11: "No Me Siento Igual"
Song 12: "Na’ De Ti"
Song 13: "Despacio (Sensei-Session)"
Song 14: "DALE"
Song 15: "Tómalo"
Song 16: "Estamos Mal"
Song 17: "La Oportunidad"
Song 18: "Las Justas 2"
Song 19: "Ahora Dice (Sensei-Session)"
Song 20: "¿Donde?"
Song 21: "Si Me Permites"
Song 22: "Química*"
Song 23: "Nunca Me Olvidaras"
Song 24: "No Hay Nadie Mas"
Song 25: "De Cora ﹤3*"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searchin

Searching for songs by Five Finger Death Punch...

Song 1: "Wrong Side of Heaven"
Song 2: "Gone Away"
Song 3: "Jekyll and Hyde"
Song 4: "Sham Pain"
Song 5: "Blue on Black"
Song 6: "I Refuse"
Song 7: "A Little Bit Off"
Song 8: "House of the Rising Sun"
Song 9: "Fake"
Song 10: "I Apologize"
Song 11: "When the Seasons Change"
Song 12: "Remember Everything"
Song 13: "Far from Home"
Song 14: "Wash it All Away"
Song 15: "Coming Down"
Song 16: "Bad Company"
Song 17: "The Bleeding"
Song 18: "The Pride"
Song 19: "Trouble"
Song 20: "Inside Out"
Song 21: "Lift Me Up"
Song 22: "Under and Over It"
Song 23: "Top of the World"
Song 24: "Darkness Settles In"
Song 25: "Will the Sun Ever Rise"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by Manuel Turizo...

Song 1: "Culpables"
Song 2: "Esperándote"
Song 3: "Quiéreme Mientras Se Pueda"
Song 4: "Una Lady Como Tú"
Song 5: "Una Lady Como Tú (Remix)"
Song 6: "Nada Ha Cambiado"
Song 7: "Sola"
Song 8: "Caso Perdido"
Song 

Song 10: "Thin Line"
Song 11: "Jimmy Iovine"
Song 12: "Neon Cathedral"
Song 13: "A Wake"
Song 14: "Starting Over"
Song 15: "Gold"
Song 16: "Castle"
Song 17: "Cowboy Boots"
Song 18: "Kevin"
Song 19: "And We Danced"
Song 20: "Make the Money"
Song 21: "My Oh My"
Song 22: "Otherside (Remix)"
Song 23: "Need to Know"
Song 24: "Victory Lap"
Song 25: "Light Tunnels"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by Galantis...

Song 1: "Peanut Butter Jelly"
Song 2: "No Money"
Song 3: "Runaway (U & I)"
Song 4: "Hunter"
Song 5: "Bones"
Song 6: "True Feeling"
Song 7: "Mama Look at Me Now"
Song 8: "Rich Boy"
Song 9: "Gold Dust"
Song 10: "Firebird"
Song 11: "Satisfied"
Song 12: "In My Head"
Song 13: "Spaceship"
Song 14: "You"
Song 15: "San Francisco"
Song 16: "Emoji"
Song 17: "Holy Water"
Song 18: "Help"
Song 19: "Pillow Fight"
Song 20: "Written in the Scars"
Song 21: "Louder, Harder, Better"
Song 22: "Call If You Need Me"
Song 23: "Hey Alligator"
Song 24: "Smile

Song 1: "Let It Go"
Song 2: "Us"
Song 3: "Hold Back the River"
Song 4: "Pink Lemonade"
Song 5: "Peer Pressure"
Song 6: "Scars"
Song 7: "Wild Love"
Song 8: "Bad"
Song 9: "If You Ever Want To Be In Love"
Song 10: "Slide"
Song 11: "Need the Sun to Break"
Song 12: "Move Together"
Song 13: "Incomplete"
Song 14: "Break My Heart Right"
Song 15: "Chew On My Heart"
Song 16: "Wasted on Each Other"
Song 17: "I Found You"
Song 18: "Best Fake Smile"
Song 19: "Just for Tonight"
Song 20: "Craving"
Song 21: "Wanderlust"
Song 22: "Fade Out"
Song 23: "In My Head"
Song 24: "Stand Up"
Song 25: "When We Were On Fire"

Reached user-specified song limit (25).
Done. Found 25 songs.
Searching for songs by Don Omar...

Song 1: "That’s How We Roll (Fast Five Remix)"
Song 2: "Guaya Guaya"
Song 3: "Dile"
Song 4: "Dutty Love"
Song 5: "Salió El Sol"
Song 6: "Pobre Diabla"
Song 7: "Virtual Diva"
Song 8: "Dale Don Dale"
Song 9: "Conteo"
Song 10: "How We Roll (Fast Five Remix)"
Song 11: "Taboo"
Song 12: "Hasta Abajo (R